<h1><center>Needleman-Wunsch</center></h1>


On va donc créer l'algorithme Needleman-Wunsch qui effectue un alignement global. Le but est donc de trouver l'alignement de score maximal.

In [1]:
import numpy as np
import pandas as pd
import random
import math
from Bio.SubsMat import MatrixInfo

La fonction suivante est créée car dans l'import des matrices BLOSUM, les dico ne contiennent pas les couples à l'envers,
on a AB et non AA par exemple.

In [2]:
def score_pair(a, b, matrix):
    """
    retourne le score d'un couple d'aa peu importe l'ordre des aa (on les inverse si besoin)
    :param a
    :param
    :param
    :return
    """
    if (a, b) not in matrix:
        return matrix[(b, a)]
    else:
        return matrix[(a, b)]

In [3]:
def show_matrix(matrix, seq1, seq2):
    '''
    affiche le contenu de matrix avec les header: seq1 verticalement et seq2 horizontalement
    :param matrix: une matrice à afficher
    :param seq1: string 
    :param seq2: string
    :return: None
    '''
    # Convertit les chaînes de caractères en listes et ajoute un espace au début à cause de la ligne et la colonne d'init
    line = list(seq1)
    line.insert(0, ' ')
    column = list(seq2)
    column.insert(0, ' ')
    # Print the dataframe with the previous line and column
    print(pd.DataFrame(matrix, index=line, columns=column))

In [4]:
def create_matrix(seq1, seq2, gap_penalty, dict_vals):
    """
    crée la matrice remplie des scores 
    :param seq1: sequence 1 -> string
    :param seq2: sequence 2 -> string
    :param gap_penalty: un entier
    :param dict_vals: un dico correspondant à la matrice de substitution
    :return nw
    """
    line = list(seq1)
    line.insert(0, ' ')
    column = list(seq2)
    column.insert(0, ' ')
    
    # On créé la matrice qui est de la taille de nos sequences plus un (ligne et colonne d'initialisation)
    nw = np.zeros((len(seq1)+1, len(seq2)+1), dtype=np.int)
    # On remplit la première ligne et la première colonne
    for i in range(nw.shape[0]):
        nw[i][0] = gap_penalty * i
    for i in range(nw.shape[1]):
        nw[0][i] = gap_penalty * i
    
    # On remplit le reste de la matrice
    for i in range(1, nw.shape[0]):
        for j in range(1, nw.shape[1]):
            nw[i][j] = max(nw[i-1][j] + gap_penalty,  # case du dessus plus pénalité
                           nw[i][j-1] + gap_penalty,  # case de gauche plus pénalité
                           nw[i-1][j-1] + score_pair(line[i], column[j], dict_vals))  # case de diagonale (supérieure
                                                                                      # gauche) plus le score récupéré
                                                                                      # dans notre dictionnaire
    # On retourne la matrice que l'on vient de créer
    return nw

In [5]:
def get_track(matrix):
    """
    permet de faire le track-back dans la matrice afin d'enregistrer le chemin
    :param matrix: la matrice remplie des scores
    :return track_back est une liste de couples de tuples
    """
    i, j = matrix.shape[0]-1, matrix.shape[1]-1  # Initialisation des indices en bas et à droite de la matrice
    track_back = [(i, j)]  # Insertion du dernier élément en bas à droite
    while i != 0 and j != 0:
        if i == 0:  # Si on conserve la ligne...
            j -= 1  # on décrémente seulement l'indice de la colonne
        elif j == 0:  # Si on conserve la colonne
            i -= 1    # on décrémente seulement l'indice de la ligne
        else:                           
            left = matrix[i][j-1]
            right = matrix[i-1][j]
            if left + gap_penalty == matrix[i][j]:
                j -= 1
            elif right + gap_penalty == matrix[i][j]:
                i -= 1
            else:
                i -= 1
                j -= 1
        track_back.insert(0, (i, j))  # Insertion du nouvel élément en tête de notre liste
    track_back.insert(0, (0, 0))  # Insertion du premier élément en haut à gauche
    
    return track_back

In [6]:
def show_result(trace, seq2, seq1):
    """
    permet d'afficher les résultats de l'alignement sur 3 lignes
    :param trace: liste de couples de tuples représentant le chemin
    :param seq1 -> string
    :param seq2 -> string
    :return None
    """
    # Définition des pipes avec couleurs pour l'affichage, \x1b[0m correspond à noir et permet de réinitialiser la couleur  
    pipe_green, pipe_blue, pipe_red = '\x1b[32m|\x1b[0m', '\x1b[34m|\x1b[0m', '\x1b[31m|\x1b[0m'
    # Initialisation des lignes que l'on affichera
    line1, line2, line3 = [], [], []
    # Initialistion des indices correspondant aux aa déjà utilisés dans nos séquences
    next_char_in_seq1 = 0
    next_char_in_seq2 = 0
    
    # Pour chaque couple d'indices dans notre trace
    for index in range(len(trace)-1):
        # On récupère les valeurs actuelles lignes et colonnes...
        i1, j1 = trace[index]
        # ... ainsi que les valeurs précédentes
        i2, j2 = trace[index+1]
        if i1 == i2:  # Si la ligne est inchangée, on introduit un gap dans la colonne (seq2)
            line1.append(seq1[next_char_in_seq1])  # On ajoute l'aa à la ligne (seq1)
            line2.append(pipe_green)               # On ajoute le pipe de couleur verte
            line3.append('-')                      # On introduit le gap
            next_char_in_seq1 += 1                 # On incrémente la liste d'aa disponible pour la ligne
        elif j1 == j2: # Si la colonne est inchangée, on introduit un gap dans la ligne (seq1)
            line1.append('-')                      # On introduit le gap
            line2.append(pipe_red)                 # On ajoute le pipe rouge
            line3.append(seq2[next_char_in_seq2])  # On ajoute l'aa à la colonne (seq2)
            next_char_in_seq2 += 1                 # On incrémente la liste d'aa disponible pour la colonne
        else:          # Sinon, colonne et ligne changent
            line1.append(seq1[next_char_in_seq1])  # On ajoute l'aa à la ligne (seq1)
            line2.append(pipe_blue)                # Ajout du pipe bleu
            line3.append(seq2[next_char_in_seq2])  # On ajoute l'aa à la colonne (seq2)
            next_char_in_seq1 += 1                 # On incrémente la liste d'aa disponible pour la ligne
            next_char_in_seq2 += 1                 # On incrémente la liste d'aa disponible pour la colonne
    
    # Affiche des lignes crééent précedemment avec un espace entre les caractères
    print(' '.join(line1))
    print(' '.join(line2))
    print(' '.join(line3))

In [7]:
def compute_z(score_reel, score_list_random_sequence):
    """
    Permet de calculer le z-score
    :param score_reel (dernière ligne, dernière colonne) -> int
    :param score_list_random_sequence est une liste de scores obtenue avec le shuffle des 2 séquences
    :return Z-score
    """
    mu = sum(score_list_random_sequence)/len(score_list_random_sequence)
    sigma = 0
    for s in score_list_random_sequence:
        sigma += (s - mu)**2
    sigma = math.sqrt((1/len(score_list_random_sequence))*sigma)
    
    return (score_reel - mu)/sigma

# programme (main)

In [19]:
# Original
seq_input_1 = 'GGVTTF'
seq_input_2 = 'MGGETFA'
gap_penalty = -4
blosum = MatrixInfo.blosum62  #on peut changer le 62 par 50 ou autre
nw = create_matrix(seq_input_1, seq_input_2, gap_penalty, blosum)
show_matrix(nw, seq_input_1, seq_input_2)

print()
trace = get_track(nw)

show_result(trace, seq_input_1, seq_input_2)

# Shuffle
list_of_score_for_random_sequences = []
for _ in range(10000):
    seq_input_1_shuffle = ''.join(random.sample(seq_input_1,len(seq_input_1)))
    seq_input_2_shuffle = ''.join(random.sample(seq_input_2,len(seq_input_2)))
    nw_shuffle = create_matrix(seq_input_1_shuffle, seq_input_2_shuffle, gap_penalty, blosum)

    list_of_score_for_random_sequences.append(nw_shuffle[nw_shuffle.shape[0]-1][nw_shuffle.shape[1]-1])

z = compute_z(nw[nw.shape[0]-1][nw.shape[1]-1], list_of_score_for_random_sequences)

print()
print('z_score =', z)

        M   G   G   E   T   F   A
    0  -4  -8 -12 -16 -20 -24 -28
G  -4  -3   2  -2  -6 -10 -14 -18
G  -8  -7   3   8   4   0  -4  -8
V -12  -7  -1   4   6   4   0  -4
T -16 -11  -5   0   3  11   7   3
T -20 -15  -9  -4  -1   8   9   7
F -24 -19 -13  -8  -5   4  14  10

M G G - E T F A
| | | | | | | |
- G G V T T F -

z_score = 1.2682783701546312


On se rend compte que si on utilise une BLOSUM50 car on a 50% d'identité entre les deux séquences, le z_score s'améliore et passe à 1.44 (plus grand). En effet, Le score réel doit être le plus grand possible puisque c'est le but de l'algorithme et il doit donc être supérieur à la moyenne des scores d'alignement de sequences mélangées. Pour cette raison, notre z_score doit être le plus grand possible pour être le meilleur possible puisque la formule est z_score= (score réel - moy des scores des sequences mélangées)/écart type. Evidemment,nos séquences sont courtes et un pourcentage d'identité sur des séquences aussi courtes est peu significatif ce qui ici ne permet pas de voir de gros écart de z_score en changeant de matrice de substitution. Si on change la pénalité de gap en mettant une valeur positive (ça n'a pas de sens biologique, je sais), on voit un alignement différent avec plus de gaps, ces derniers étant très favorisés et un z_score supérieur. Si la gap pénalité est de -10, on a un seul gap au lieu de deux et un z-score inférieur. Il ne faut donc pas avoir une pénalité de gap positive car ça n'a pas de sens biologique bien que le z_score soit augmenté et en diminuant la pénalité, on diminue le z_score ce qui n'est pas non plus bon. 